In [ ]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
# import snap
import random
from numpy.random import choice
import numpy as np
from collections import defaultdict
from tqdm import tnrange
import scipy.sparse
# from playlist import *
import playlist
reload(playlist)

In [ ]:
playlist_list = playlist.get_playlist_list(50)

In [ ]:
len(playlist_list)

In [ ]:
track_list = set([])
for i in range(len(playlist_list)):
    for j in range(playlist_list[i]['num_tracks']):
        track_uri = playlist_list[i]['tracks'][j]['track_uri']
        track_list.add(track_uri)
track_list = list(sorted(track_list))
track_order_dict = dict(zip(track_list, range(len(track_list))))

In [ ]:
len(track_list)

In [ ]:
# feature_matrix = np.zeros((len(playlist_list), len(track_list)))
# feature_matrix.shape

In [ ]:
feature_matrix = scipy.sparse.lil_matrix((len(playlist_list), len(track_list)))
feature_matrix.shape

In [ ]:
for i in range(len(playlist_list)):
    for j in range(playlist_list[i]['num_tracks']):
        track_uri = playlist_list[i]['tracks'][j]['track_uri']
        k = track_order_dict[track_uri]
        feature_matrix[i,k] = 1    

In [ ]:
# if 1 in feature_matrix[:, :]:
#     print"1"

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=100, init='random', random_state=0, verbose=True)
W = model.fit_transform(feature_matrix)
H = model.components_

In [ ]:
model.reconstruction_err_

In [ ]:
import gzip
import pickle

with gzip.open('track_H_50k_100n.pickle.gz','wb') as output:
    pickle.dump(H ,output)

In [ ]:
with gzip.open("track_H_50k_100n.pickle.gz", "rb") as inp:
    H = pickle.load(inp)

In [ ]:
W.shape

In [ ]:
H.shape

In [ ]:
(H.T).shape

In [ ]:
track_H_df = pd.DataFrame(data=H.T, index = track_list)

In [ ]:
track_H_df.head()

In [ ]:
track_H_df = playlist.normalize_dataframe(track_H_df)
track_H_df.head()

In [ ]:
def get_track_embedding(track_uri):
    return track_H_df.loc[track_uri].values

In [ ]:
cached_playlist_embedding = {}

def get_playlist_embedding(playlist_id):
    # check if already in cache 
    if playlist_id in cached_playlist_embedding:
        return cached_playlist_embedding[playlist_id]

    embedding_list = []
    for i in range(playlist_list[playlist_id]['num_tracks']):
        track_uri = playlist_list[playlist_id]['tracks'][i]['track_uri']
    #     print(type(track_df.loc[track_uri]))
        embedding_list.append(get_track_embedding(track_uri))
    embedding = np.mean(embedding_list, axis = 0)
    # put in the cache
    cached_playlist_embedding[playlist_id] = embedding
    return embedding

In [ ]:
for playlist_id in tnrange(len(playlist_list)):
    get_playlist_embedding(playlist_id)

In [ ]:

def cal_similarity(playlist_id_1, playlist_id_2):
    return np.inner(cached_playlist_embedding[playlist_id_1],
                    cached_playlist_embedding[playlist_id_2])
#         similarity = sklearn.metrics.pairwise.cosine_similarity(get_playlist_embedding(playlist_id_1).reshape(1, -1), 
#                                                                 get_playlist_embedding(playlist_id_2).reshape(1, -1))
#     similarity = sklearn.metrics.pairwise.euclidean_distances(get_playlist_embedding(playlist_id_1).reshape(1, -1), 
#                                                                 get_playlist_embedding(playlist_id_2).reshape(1, -1))
#         return similarity[0][0]
#     return -similarity[0][0]

In [ ]:
def biased_H_random_walk_playlist(query,g,dropped_track_id, beta = 1):
    
    N = 1000
    totSteps = 0
    nHighVisited = 0
    nt = 200
    nv = 5
    num_visits = defaultdict(int)
    
    weight_cache = {}
    def cal_weight(edge):
        if edge in weight_cache:
            return weight_cache[edge]

        val = cal_similarity(query,edge)
        weight_cache[edge] = val
        return val

    while totSteps < N or nHighVisited > nt: 
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            edges = list(g[currPlaylist])
            if currPlaylist == query:
                edges.remove(dropped_track_id)
            currTrack = random.choice(edges)
            # takes 1 step (from a track to playlist)
            edges = list(g[currTrack])
            if currTrack == dropped_track_id:
                edges.remove(query)
#             weights = [f(edge) for edge in edges]
            weights = [cal_weight(edge) for edge in edges]
            weights_np = np.asarray(weights)

# uncomment to do softmax
# see https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
#             # this bit makes it more stable numerically
            max_weight = np.max(weights_np)
            weights_np -= max_weight
            # but does not change the answer
            weights_np = np.exp(beta * weights_np)

            prob_dis = weights_np / float(sum(weights_np))
            currPlaylist = int(choice(edges, 1, p=prob_dis))
            num_visits[currTrack] += 1
            if num_visits[currTrack] == nv:
                nHighVisited += 1
        totSteps += currSteps
    return num_visits

In [ ]:
g, tracks_dict, tracks_id_dict = playlist.make_graph_dict(playlist_list)

In [ ]:
def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_id = random.choice(tuple(g[playlist_id]))

    if len(g[playlist_id]) < 5:
        return None, None
    num_visits = biased_H_random_walk_playlist(playlist_id, g, dropped_track_id)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

In [ ]:
# all_true_ids, all_suggested_ids = cal_most_visited(playlist_list)
all_true_ids, all_suggested_ids = playlist.process_playlists(cal_most_visited_playlist, playlist_list)

In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids)
filename = 'results%d_collabrative_filtering.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))

In [ ]:
len(tracks_dict)

In [ ]:
tracks_dict, tracks_id_dict = playlist.make_tracks_dict(playlist_list)

In [ ]:
track_df = track_H_df.loc[tracks_dict.keys()]
track_array = track_df.values


In [ ]:
track_df.shape

In [ ]:
def make_query(tracks, drop_track_index, track_df, track_array):
    track_indices = [track_df.index.get_loc(track_uri)
                     for i, track_uri in enumerate(tracks) if i != drop_track_index]
    return np.mean(track_array[track_indices, :], 0)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tnrange
import random

def cal_knn(k, track_array, track_df, tracks_dict):
    nbrs = NearestNeighbors(n_neighbors=k, metric='cosine', n_jobs=1).fit(track_array)
    all_true_ids = []
    all_distances = []
    all_suggested_ids = []

    for i in tnrange(len(playlist_list)):
        tracks = playlist.get_tracks(playlist_list[i])
        if len(tracks) < 5:
            continue
        dropped_track_index = random.choice(range(len(tracks)))
        true_track_uri = tracks[dropped_track_index]
        true_id = tracks_dict[true_track_uri]
        all_true_ids.append(true_id)

        query = make_query(tracks, dropped_track_index, track_df, track_array)
        distances, suggested_index_array = nbrs.kneighbors(query.reshape(1,-1))
        suggested_ids = [tracks_dict[track_df.index[index]]
                         for index in suggested_index_array.flatten()]
        all_distances.append(distances)
        all_suggested_ids.append(suggested_ids)
    return all_true_ids, all_distances, all_suggested_ids

In [ ]:
all_true_ids, all_distances, all_suggested_ids = cal_knn(500, track_array, track_df, tracks_dict)

In [ ]:
np.savez("ids_collabrative_filtering_50k", all_true_ids=all_true_ids,
         all_distances=all_distances, all_suggested_ids = all_suggested_ids)

In [ ]:
x=np.load("ids_collabrative_filtering_50k.npz")
all_true_ids=x['all_true_ids']
all_suggested_ids = x['all_suggested_ids']

In [ ]:
playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids.tolist())

In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids)
filename = 'results%d_collabrative_filtering_knn.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))